<a href="https://colab.research.google.com/github/LeeGitaek/-_-/blob/master/KNN%2Bdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

iris 데이터셋 불러오기


In [1]:
import seaborn as sns
iris = sns.load_dataset('iris')
print(iris.head())

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
print(iris.shape)
X = iris.drop('species',axis = 1)
print(X.shape)
y=iris['species']

(150, 5)
(150, 4)


카테고리의 실수화 

In [5]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
classic = LabelEncoder()
y = classic.fit_transform(iris['species'].values)
print('species labels :',np.unique(y))

species labels : [0 1 2]


In [6]:
yo = classic.inverse_transform(y) # 원래의 species 문자열로 전환 
print('species:',np.unique(yo))

species: ['setosa' 'versicolor' 'virginica']


데이터 분할 

In [7]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=1, stratify=y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(105, 4)
(45, 4)
(105,)
(45,)


모형 추정 및 사례중심 학습


In [8]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5,p=2)
#5개의 인접한 이웃, 거리측정기준 : 유클리드 
#knn.fit
knn.fit(X_train,y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [16]:
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test) #모델을 적용한 test data dml y값 예측치 
print('Misclassified training samples : %d' %(y_train!=y_train_pred).sum())
# 오분류 데이터 갯수 확인 
print('Misclassified test samples : %d' %(y_test!=y_test_pred).sum())

Misclassified training samples : 2
Misclassified test samples : 1


In [17]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_test_pred))
# 싸이킷런 제공 분류 성능 평가 방법 
#confusion_matrix
#accuracy_score
#precision_score
#recall_score
#fbeta_score
#f1_score
#roc_curve
#auc

0.9777777777777777


In [18]:
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_true=y_test,y_pred=y_test_pred)
#confusion matrix : 혼합행렬 => 타겟의 원래 클래스와 모형이 예측한 클래스가 일치하는지
#갯수로 센 결과를 표로 나타낸 것.
print(conf)

[[15  0  0]
 [ 0 15  0]
 [ 0  1 14]]


(옵션) 입력데이터의 표준화 

In [19]:
#표준화 : 특성자료의 측정 단위에 의해 영향 받지 않도록 하는 과정
#시험 데이터의 표준화는 학습 데이터에서 구한 특성 변수의 평균과 표준편차를 이용함.
#표준화로 인해 데이터의 분포인 통계적 특성이 깨지면 머신러닝의 학습 저하를 가져옴  
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

#표준화된 data 의 확인 
print(X_train.head())
X_train_std[1:5,]


     sepal_length  sepal_width  petal_length  petal_width
33            5.5          4.2           1.4          0.2
20            5.4          3.4           1.7          0.2
115           6.4          3.2           5.3          2.3
124           6.7          3.3           5.7          2.1
35            5.0          3.2           1.2          0.2


array([[-0.55053619,  0.76918392, -1.16537974, -1.30728421],
       [ 0.65376173,  0.30368356,  0.84243039,  1.44587881],
       [ 1.0150511 ,  0.53643374,  1.0655204 ,  1.18367281],
       [-1.03225536,  0.30368356, -1.44424226, -1.30728421]])

In [20]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5,p=2) #5개의 인접한 이웃, 거리측정 기준 유클리드
knn.fit(X_train_std,y_train) # 모델 fitting 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [21]:
y_train_pred = knn.predict(X_train_std)
y_test_pred = knn.predict(X_test_std)

print('Misclassified training samples : %d' %(y_train!=y_train_pred).sum())
print('Misclassified test samples : %d' %(y_test!=y_test_pred).sum())


Misclassified training samples : 4
Misclassified test samples : 3


In [22]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_test_pred))
# 표준화로 인해 정확도가 97.8 에서 93.3 으로 떨어진 사례이다. 
# 표준화 여부는 시험 데이터와 정밀도를 점검하여 결정하도록 한다. 

0.9333333333333333


KNN 회귀를 이용한 영화 평점 예측 

In [26]:
from sklearn.neighbors import KNeighborsRegressor

regressor = KNeighborsRegressor(n_neighbors=3,weights = "distance")
training_points = [
    [0.5,0.2,0.1],
    [0.9,0.7,0.3],
    [0.4,0.5,0.7]
]

training_labels = [5.0,6.8,9.0]
regressor.fit(training_points, training_labels)

unknown_points = [
    [0.2,0.1,0.7],
    [0.4,0.7,0.6],
    [0.5,0.8,0.1]
]

guesses = regressor.predict(unknown_points)
guesses

array([7.28143288, 7.76451922, 6.8457845 ])